WARNING : MAKE SURE YOU GO THROUGH pytorch_env_install.ipynb IN ORDER TO INSTALL PIPENV AND DEPENDENCIES
Project done on VisualStudio Code, will automatically prompt for necessary extensions
YOU NEED TO INSTALL PYTHON 3.7.9 ON YOUR SYSTEM !

Function definitions

In [10]:
from librarybuilding_utils import tipettNoiser_fromARRAY
import os
import numpy as np
from PIL import Image
import re
import random
import csv

"""
Defining file variables
"""
main_dir = "./data2"
original_images_folder = "/og_data"
train_directory_clean_clones = "/train_set/train_noiseless_clones"
train_directory_noisy_clones = "/train_set/train_noisy_clones"
test_directory_clean_clones = "/test_set/test_noiseless_clones"
test_directory_noisy_clones = "/test_set/test_noisy_clones"

def createIfNotExisting():
    if not (os.path.exists(main_dir+train_directory_clean_clones)):
        os.makedirs(main_dir+train_directory_clean_clones)
        print(f"created new directory {main_dir+train_directory_clean_clones}")
    if not (os.path.exists(main_dir+train_directory_noisy_clones)):
        os.makedirs(main_dir+train_directory_noisy_clones)
        print(f"created new directory {main_dir+train_directory_noisy_clones}")
    if not (os.path.exists(main_dir+test_directory_clean_clones)):
        os.makedirs(main_dir+test_directory_clean_clones)
        print(f"created new directory {main_dir+test_directory_clean_clones}")
    if not (os.path.exists(main_dir+test_directory_noisy_clones)):
        os.makedirs(main_dir+test_directory_noisy_clones)
        print(f"created new directory {main_dir+test_directory_noisy_clones}")

def array_divider(img,sqr_size):
    height,width = img.shape

    print(f"height = {height}, width = {width}")

    hrz = 0
    vrt = 0
    slices = []

    while(vrt+sqr_size<width):
        while(hrz+sqr_size<height):
            newslice = img[vrt:vrt+sqr_size,hrz:hrz+sqr_size]
            #If condition so that we dont have non square slices
            if(newslice.shape[0] == newslice.shape[1]):
                slices.append(img[vrt:vrt+sqr_size,hrz:hrz+sqr_size])   

            hrz+=sqr_size
        hrz=0
        vrt+=sqr_size
    
    return slices

def multi_flipper(array):
    return np.array([np.flip(array, 0), np.flip(array, 1),np.flip(np.flip(array, 0),1)])

def multi_rotate(array):
    return np.array([np.rot90(array,1,(0,1)),np.rot90(array,2,(0,1)),np.rot90(array,3,(0,1))])

def flipper(array,index):
    if(index==2):
        return np.flip(np.flip(array, 0),1)
    return np.flip(array, index)

def rotate(array, index):
    return np.rot90(array,index,(0,1))

#VERSION TROP GOUrMANDE EN MEMOIRE : ARRAY CONTENANT LES CLONES TROP VOLUMINEUX
def getClones(img,sqr_size):
    slices = array_divider(img,sqr_size)
    clones = slices
    for slice in slices:
        tmp_flipped_slices = multi_flipper(slice)
        for flipped_slice in tmp_flipped_slices:
            clones.append(flipped_slice)
    
    clones_rot = clones
    for clone in clones:
        tmp_rot_slices = multi_rotate(clone)
        for rot_slice in tmp_rot_slices:
            clones_rot.append(rot_slice)

    clones_noisy = np.array([])
    for clone in clones_rot :
        np.append(clones_noisy,tipettNoiser_fromARRAY(clone,3,1.3))

    return clones,clones_noisy

#UTILISATION DE YIELD POUR ECRIRE LES IMAGES UNE PAR UNE SUR LE DISQUE
def getClones_yield(img,sqr_size):        
    slices = array_divider(img,sqr_size)
    for slice in slices:
        for rot_index in [1,2,3]:
            for flip_index in [0,1,2]:

                clean_clones = rotate(flipper(slice,flip_index),rot_index)

                noisy_clones = tipettNoiser_fromARRAY(clean_clones,1,1.3)
                yield clean_clones,noisy_clones


def createData_yield():

    directory = os.fsencode(main_dir+original_images_folder)
    
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        print(filename)

        og_img=np.load(os.fsdecode(directory)+"/"+filename)

    
        clones = getClones_yield(og_img,256)
        index = 0
        for clone in clones:
            index+=1

            #first member of the tuple is the rotate+flipper noise-FREE slice
            PILimg = Image.fromarray(clone[0])
            PILimg.convert("L").save(main_dir+train_directory_clean_clones+"/noiseless_"+filename[:len(filename) -4]+"_"+str(index)+".png")

            #first member of the tuple is the rotate+flipper noisY slice
            PILimg = Image.fromarray(clone[1])
            PILimg.convert("L").save(main_dir+train_directory_noisy_clones+"/noisy_"+filename[:len(filename) -4]+"_"+str(index)+".png")

"""
This is a one shot execution script moving a fraction of the training database into the test
database, following the "test_ratio" proportion
Execute after running createData_yield()
"""
test_ratio = 0.1

def sort_data_set():
    to_be_moved = []
    
    for file in os.listdir(main_dir+train_directory_clean_clones):
        filename = os.fsdecode(file)
        if(random.random()<test_ratio):
            #print(re.sub("noiseless_", "", filename, count=0, flags=0))
            to_be_moved.append(re.sub("noiseless_", "", filename, count=0, flags=0))

    for filename in to_be_moved:
        os.replace(main_dir+train_directory_clean_clones+"/noiseless_"+filename, main_dir+test_directory_clean_clones+"/noiseless_"+filename)
        os.replace(main_dir+train_directory_noisy_clones+"/noisy_"+filename, main_dir+test_directory_noisy_clones+"/noisy_"+filename)

"""
Creates a csv file that maps the original SAR images to their noisy clone
"""

def create_dataset_csv():
    name_couples_test = {}    
    for file in os.listdir(main_dir+"/test_set"+"/test_noiseless_clones"):
        filename = os.fsdecode(file)
        #print(re.sub("noiseless_", "", filename, count=0, flags=0))
        name_couples_test[filename]="noisy_"+re.sub("noiseless_", "", filename, count=0, flags=0)

    name_couples_train = {}    
    for file in os.listdir(main_dir+"/train_set"+"/train_noiseless_clones"):
        filename = os.fsdecode(file)
        #print(re.sub("noiseless_", "", filename, count=0, flags=0))
        name_couples_train[filename]="noisy_"+re.sub("noiseless_", "", filename, count=0, flags=0)

    with open(main_dir+'/dataset_mapping_test.csv', mode='w') as map_test:

        fieldnames = ['noiseless', 'noisy']
        writer = csv.DictWriter(map_test, fieldnames=fieldnames)

        writer.writeheader()

        for key in name_couples_test:
            writer.writerow({fieldnames[0]:key, fieldnames[1]:name_couples_test[key]})

    with open(main_dir+'/dataset_mapping_train.csv', mode='w') as map_test:

        fieldnames = ['noiseless', 'noisy']
        writer = csv.DictWriter(map_test, fieldnames=fieldnames)

        writer.writeheader()

        for key in name_couples_train:
            writer.writerow({fieldnames[0]:key, fieldnames[1]:name_couples_train[key]})

outside_images_folder = "/outside_data"
outside_directory_clean_clones = "/outside_set"

"""
Prepares images for usage through our model

"""
def getOutsideClones(outsideImg, sqr_size):
    slices = array_divider(outsideImg,sqr_size)
    for slice in slices:
        yield slice

"""
Prepares images for usage through our model

"""
def prepareOutsideData_yield():

    directory = os.fsencode(main_dir+outside_images_folder)
    
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        print(filename)

        og_img=np.load(os.fsdecode(directory)+"/"+filename)

    
        clones = getOutsideClones(og_img,256)
        index = 0
        for clone in clones:
            index+=1

            #first member of the tuple is the rotate+flipper noise-FREE slice
            PILimg = Image.fromarray(clone)
            PILimg.convert("L").save(main_dir+outside_directory_clean_clones+"/outside_"+filename[:len(filename) -4]+"_"+str(index)+".png")

"""
Creates a csv file with the file names list of "outside" pictures

"""
def create_outside_dataset_csv():
    name_couples_outside = {}    
    for file in os.listdir(main_dir+outside_directory_clean_clones):
        filename = os.fsdecode(file)
        #print(re.sub("noiseless_", "", filename, count=0, flags=0))
        name_couples_outside[filename]=filename

    with open(main_dir+'/dataset_mapping_outside.csv', mode='w') as map_test:

        fieldnames = ['noiseless', 'noisy']
        writer = csv.DictWriter(map_test, fieldnames=fieldnames)

        writer.writeheader()

        for key in name_couples_outside:
            writer.writerow({fieldnames[0]:key, fieldnames[1]:name_couples_outside[key]})

Database building

In [11]:
"""
Ensure the directories are ready

"""
createIfNotExisting()

In [7]:
"""
Creates sliced clones (256x256 by default) of original images and puts them in the "./data2/train_set/  train_noiseless_clones and train_noisy_clones" folders

"""
createData_yield()

Alpes1_21.npy
height = 1076, width = 1076
Damoh_21.npy
height = 1026, width = 1026
DesMoines_21.npy
height = 2143, width = 2795
Rennes_26.npy
height = 1931, width = 1159


In [3]:
"""
Moves a fraction of the train set to the test set (default=10%)

"""
sort_data_set()

In [4]:
"""
Creates a csv file with the file names list of "train" and "test" pictures for easier dataset loading once in torch's models

"""
create_dataset_csv()

In [5]:
"""
Prepares images for usage through our model

"""
prepareOutsideData_yield()

lely.npy
height = 500, width = 500
marais1.npy
height = 500, width = 500


In [6]:
"""
Creates a csv file with the file names list of "outside" pictures

"""
create_outside_dataset_csv()